# **Homework 8 - Anomaly Detection**

If there are any questions, please contact mlta-2023-spring@googlegroups.com

Slide:    [Link](https://docs.google.com/presentation/d/18LkR8qulwSbi3SVoLl1XNNGjQQ_qczs_35lrJWOmHCk/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/t/c76950cc460140eba30a576ca7668d28)

# Auto-encoder 
## Pre-train = Self-supervised Learning不需要label data   Fine-tuning一下就可以用在下游
## Encoder(Dimension Reduction):vector to vector,输出Embedding,Representation,Code,bottleneck(非常低维的向量).图片是一个高维度的向量.
## Decoder:vector to vector
## 希望Encoder的输入和Decoder的输出(reconstruction)尽可能相似
## 架构很像GAN,多层Network,就是Cycle GAN

### De-noising Auto-encoder,多了一个任务:去掉noises.和BERT一样

## Feature Disentangle:把Embedding的维度和它们对应的资讯对应起来
## 应用于Voice Conversion,Encoder的输出为内容＋特征,两者拼起来丢入Decoder

## Discrete Representation:Embedding是Binary,是One-hot vector,可以做到Unsupervised的分类.  
### VQVAE:和Self-Attention很像,Query和codebook里面的Attention比较相似度
### Text as Representation:文字丢入En,生成word sequence,是文章的Summary,输入De产生Document.En和De都是Seq2seq. 实际上是行不通的.因为产生的Summary is not readable... 所以要加入一个Discriminator(Human written summaries),判断(Real or not).
### 其实就变成了Cycle GAN,只不过从Auto-Encoder的角度看Cycle GAN
# Tips:看到没办法train的问题,就用RL硬做,就结束了.
## Machine的自学能力还是挺强,所以有时候会意想不到他那里学会了新东西，有时候感觉他在胡说八道.
## Decoder有时候会被当成Generator
## Auto-encoder中的Encoder可以用来压缩,Decoder用来解压缩,但是会Lossy(失真)
## 作业为Anomaly Detection:有一些training data用来异常检测.Detecting input x is similar to training data or not.anomaly可以被称为outlier,novelty,exceptions.
## 难点在于收集资料.异常的资料太少了.所以他不是一个一般的Classification.叫One plus分类问题.Approach:Auto-encoder.

# 调整一下Model
# multi-encoder autoencoder
# add random noise and an extra classifier

In [3]:
!pip install -q qqdm
print("asdfasdf")

asdfasdf


In [16]:
import os
os.environ['KAGGLE_USERNAME'] = 'your_username'  # 替换为你的用户名
os.environ['KAGGLE_KEY'] = 'your_api_key'    # 替换为你的API key

# 然后下载比赛数据
!kaggle competitions download -c ml2023spring-hw8
print("drfqwe")

drfqwe


In [20]:
!unzip ml2023spring-hw8.zip -d /kaggle/working/

Archive:  ml2023spring-hw8.zip
replace /kaggle/working/.gitattributes? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [21]:
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
from torch.optim import Adam, AdamW
from qqdm import qqdm, format_str
import pandas as pd

print("dasasdqw")

dasasdqw


In [32]:

train = np.load('/kaggle/working/{WORK_DIR}/trainingset.npy', allow_pickle=True)
test = np.load('/kaggle/working/{WORK_DIR}/testingset.npy', allow_pickle=True)

print(train.shape)
print(test.shape)

UnpicklingError: Failed to interpret file '/kaggle/working/{WORK_DIR}/trainingset.npy' as a pickle

In [31]:
!kaggle competitions download -c ml2023spring-hw8 -f trainingset.npy -p {WORK_DIR} --force
!kaggle competitions download -c ml2023spring-hw8 -f testingset.npy -p {WORK_DIR} --force

  0%|                                                 | 0.00/135 [00:00<?, ?B/s]
100%|███████████████████████████████████████████| 135/135 [00:00<00:00, 653kB/s]
  0%|                                                 | 0.00/134 [00:00<?, ?B/s]
100%|███████████████████████████████████████████| 134/134 [00:00<00:00, 630kB/s]


# 文件头显示b'version ht'，这不是合法的.npy文件
# 很可能是Git LFS指针文件或文本文件被错误命名